## Data

For this project I have used the following datasets:

1. Seattle neighborhood data with latitude and longitude coordinates

2. Foursquare location data: Most common categories

3. Dataset on pet licences in Seattle

In the next section i will describe preliminary data importing and preparation.

Before we start, lets import all necessary libraries and setup our forsquare IDs:

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

# Setting up IDs for foursquare

CLIENT_ID = 'J2VNJFAYIYLLMUX0BP25WE1WMWUQOKRASOM0TDWHJIV0NMUG' # your Foursquare ID
CLIENT_SECRET = 'GXRQDD3VXGVRVSN4JGGPEJN1TTLSE3EMML5ZK4KTWVDCBFW2' # your Foursquare Secret
ACCESS_TOKEN = 'EW4X4KIJXWWJ5HCRXYBYL2YAYXJRBT4GAPIOTBF5LOC1ZCQF' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 79 kB 11.3 MB/s eta 0:00:01
  Created wheel for folium: file

In [2]:
# The code was removed by Watson Studio for sharing.

,Zipcode,Neighborhood
0,98003,Federal Way
1,98005,Bellevue
2,98033,Kirkland
3,98037,Lynnwood
4,98040,Mercer Island


In [3]:
# Import US zip code database

body = client_7090f49ae7784ed7b279229f4ce3b604.get_object(Bucket='helenax27sproject-donotdelete-pr-2wln2ucz6htuks',Key='zip_code_database.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df = df[['zip', 'latitude', 'longitude']]
df.rename(columns = {'zip':'Zipcode', 'latitude': 'Latitude', 'longitude':'Longitude' }, inplace=True)
df.head()

,Zipcode,Latitude,Longitude
0,501,40.81,-73.04
1,544,40.81,-73.04
2,601,18.16,-66.72
3,602,18.38,-67.18
4,603,18.43,-67.15


In order to plot the neighbourhoods on a map, i need to merge the latitude and longitude data to the Neighbourhood names. I do this my merging on the Zipcode column of both datasets:

In [4]:
neighborhoods = pd.merge(df, seattle_zips, on="Zipcode")
neighborhoods = neighborhoods.drop_duplicates(subset='Zipcode', keep="first")
neighborhoods.sort_values(by=['Neighborhood'], inplace=True)
neighborhoods.head()

,Zipcode,Latitude,Longitude,Neighborhood
20,98110,47.64,-122.53,Bainbridge Island
17,98107,47.67,-122.38,Ballard
1,98005,47.62,-122.16,Bellevue
25,98121,47.62,-122.35,Belltown
9,98102,47.63,-122.32,Capital Hill


### Foursquare location data

Now I will use the Foursquare API to identify venues in each neighborhood which I like the best. For generating the nearby venues, i have chosen a radius of 2km and limit to the top 80 venues.

In [6]:
# function to loop each neighborhood and get the 80 venues within 2km.

def getNearbyVenues(names, latitudes, longitudes, radius=2000, LIMIT = 80):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):  
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
# run the function above to get nearby venues for each neighborhood

venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                        latitudes=neighborhoods['Latitude'],
                        longitudes=neighborhoods['Longitude']
                                  )
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bainbridge Island,47.64,-122.53,Storyville Coffee Roasting Studio,47.648323,-122.523960,Coffee Shop
1,Bainbridge Island,47.64,-122.53,Fletcher Bay Winery,47.648285,-122.525015,Wine Bar
2,Bainbridge Island,47.64,-122.53,Bainbridge Island Brewing,47.648299,-122.525042,Brewery
3,Bainbridge Island,47.64,-122.53,Jake's Pickup,47.636094,-122.516740,Café
4,Bainbridge Island,47.64,-122.53,Bainbridge Island Aquatic Center,47.638323,-122.522930,Gym Pool


In [8]:
# view the size of venues dataset

ven_size = venues.shape
print('The venues dataset has {} rows and {} columns.'.format(ven_size[0], ven_size[1]))

The venues dataset has 1562 rows and 7 columns.


### Seattle pet data
For the Seattle data on pets, I have used the The Open Data Program > https://data.seattle.gov/ < generated by the City of Seattle. This makes data openly available to the public for the purpose of increasing the quality of life for residents; increasing transparency, accountability and comparability; promoting economic development and research; and improving internal performance management.

I found the following dataset avaliable: https://data.seattle.gov/Community/Seattle-Pet-Licenses/jguv-t9rb which is a list of active/current Seattle pet licenses, including animal type (species), pet's name, breed and the owner's ZIP code.

In [9]:
# Importing the dataset

body = client_7090f49ae7784ed7b279229f4ce3b604.get_object(Bucket='helenax27sproject-donotdelete-pr-2wln2ucz6htuks',Key='Seattle_Pet_Licenses.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# Leave only rows with dogs, remove NaN's and rename ZIP Code for later merging
    
pets = pd.read_csv(body)
pets.head()
pets.rename(columns = {'ZIP Code':'Zipcode'}, inplace=True)
pets = pets.dropna()
pets['Zipcode'] = pets['Zipcode'].astype(int)
pets.head()

,License Issue Date,License Number,Animal's Name,Species,Primary Breed,Secondary Breed,Zipcode
0,December 18 2015,S107948,Zen,Cat,Domestic Longhair,Mix,98117
13,May 16 2018,S126432,Chico,Cat,Domestic Shorthair,Mix,98122
15,July 10 2018,85322,Zen,Cat,Domestic Shorthair,Siamese,98105
16,August 13 2018,578368,Lucky,Cat,Oriental Shorthair,Mix,98106
17,August 30 2018,S121902,Buffy,Cat,Siamese,European Shorthair,98107
